In [1]:
from sigmaepsilon.solid.fem.cells import B2 as Beam
#from sigmaepsilon.solid.fem.cells import B3 as Beam
import numpy as np

L, d, t = 1000.0, 700.0, 10.0
E, nu = 210000.0, 0.25
F = 1000.0
n = 2  # number of element
penalty = 1e20  # penalty value

In [2]:
from linkeddeepdict import LinkedDeepDict
from linkeddeepdict.tools import getallfromkwargs
from neumann.linalg import Vector
from neumann.linalg import linspace
from neumann import repeat
from polymesh.space import StandardFrame, PointCloud, frames_of_lines
from polymesh.utils.space import index_of_closest_point, index_of_furthest_point
from polymesh.utils.topology import L2_to_L3
from sigmaepsilon.solid import Structure, LineMesh, PointData
from sigmaepsilon.solid import BeamSection

# section
section = BeamSection('CHS', d=d, t=t, n=32)
section.calculate_section_properties()
section_props = section.section_properties
A, Ix, Iy, Iz = getallfromkwargs(['A', 'Ix', 'Iy', 'Iz'], **section_props)

# material
G = E / (2 * (1 + nu))
Hooke = np.array([
    [E*A, 0, 0, 0],
    [0, G*Ix, 0, 0],
    [0, 0, E*Iy, 0],
    [0, 0, 0, E*Iz]
])

# space
GlobalFrame = StandardFrame(dim=3)

# mesh
p0 = np.array([0., 0., 0.])
p1 = np.array([0., 0., L])
coords = linspace(p0, p1, n+1)
topo = np.zeros((n, 2), dtype=int)
topo[:, 0] = np.arange(n)
topo[:, 1] = np.arange(n) + 1
if Beam.NNODE == 3:
    coords, topo = L2_to_L3(coords, topo)
i_first = index_of_closest_point(coords, np.array([0., 0., 0.]))
i_last = index_of_furthest_point(coords, np.array([0., 0., 0.]))

# essential boundary conditions
fixity = np.zeros((coords.shape[0], 6)).astype(bool)
fixity[i_first, :] = True
fixity = fixity.astype(float) * penalty

# natural boundary conditions
nodal_loads = np.zeros((coords.shape[0], 6))
nodal_loads[i_last, 0] = -F

# pointdata
pd = PointData(coords=coords, frame=GlobalFrame,
               loads=nodal_loads, fixity=fixity)

# celldata
frame = np.array([[0., 0., 1.], 
                  [0., 1., 0.], 
                  [-1., 0., 0.]])
frames = repeat(frame, topo.shape[0])
cd = Beam(topo=topo, frames=frames)

# set up mesh and structure
mesh = LineMesh(pd, cd, model=Hooke, frame=GlobalFrame)
structure = Structure(mesh=mesh)

structure.linsolve()
dofsol = structure.nodal_dof_solution(store='dofsol')

In [3]:
-F * L**3 / (3 * E * Iz), dofsol[i_last, 0]


(-0.0012460796665290223, -0.0012460796765290329)

In [4]:
F * L**2 / (2 * E * Iz), dofsol[i_last, 4]

(1.8691194997935332e-06, 1.8691195097935335e-06)